# HDF5 for reproducability

When one is doing many rounds of inference with different data and different models, it gets easy to loose track of which sampled parameters refer to which sampling round. To tackle this, we implemented a couple of utility functions and wrappers that should make it easier to keep everything together and reproducable.

In this notebook, we are going to walk through a round of sampling and we try to set everything up in a way that would allow someone else to walk in and repeat the "experiment" easily.

## Step 0: Imports & Settings

First the imports.

In [2]:
import lymph
import numpy as np
import scipy as sp 
from scipy import stats
import pandas as pd
from multiprocessing import Pool

Now some settings, e.g. the name of the HDF5 file we would later like to use.

In [3]:
demo_hdf5_file = "./_data/demo.hdf5"

## Step 1: Set up the Model

First, we will set up the model as we would normally.

In [4]:
graph = {
    ('tumor', 'primary'): ['I', 'II', 'III', 'IV'],
    ('lnl'  , 'I'):       ['II'], 
    ('lnl'  , 'II'):      ['III'],
    ('lnl'  , 'III'):     ['IV'],
    ('lnl'  , 'IV'):      []
}

In [5]:
original_model = lymph.Bilateral(graph)

In [6]:
diagnostic_spsn = {
    "MRI": [0.63, 0.81],
    "PET": [0.86, 0.79]
}
original_model.modalities = diagnostic_spsn

Generate some synthetic data we can work with later.

<div class="alert alert-info">
    Note

    This step can be skipped, as that data is already in the `./_data` directory. But it may also serve as a guide on how to generate synthetic datasets.
</div>

In [7]:
max_t = 10
t = np.arange(max_t + 1)

early_p = 0.3
late_p = 0.7

early_time_dist = sp.stats.binom.pmf(t, max_t, early_p)
late_time_dist = sp.stats.binom.pmf(t, max_t, late_p)
time_dists = {"early": early_time_dist, "late": late_time_dist}

original_model.ipsi.base_probs   = [0.05, 0.2 , 0.12, 0.1 ]
original_model.contra.base_probs = [0.01, 0.06, 0.03, 0.01]
original_model.trans_probs = [0.1, 0.3, 0.2]

In [8]:
synthetic_data = original_model.generate_dataset(
    200, [0.6, 0.4], time_dists=time_dists
)
synthetic_data.to_csv("./_data/bilateral.csv", index=False)

Now we load the data into the model instance.

In [9]:
synthetic_data = pd.read_csv("./_data/bilateral.csv", header=[0,1,2])
original_model.patient_data = synthetic_data

## Step 2: Store the model in an HDF5 file

And before we proceed any further, we store the specifics of this model instance in an HDF5 file. It will basically store the graph, the modalities with their sensitivities & specificities as well as the just loaded data in the HDF5 file and allow us to recreate an instance.

In [10]:
original_model.to_hdf5(
    filename=demo_hdf5_file, 
    groupname="original"
)

## Step 3: Prepare the sampling

In the utilities of the `lymph` package we also provide a small wrapper around the awesome [emcee](https://github.com/dfm/emcee) `EnsembleSampler` that allows us to store some inference-specific parameters before sampling and of course the samples themselves after sampling.

Let's start with the first part:

In [11]:
# plus one dimension for the late T-stage's time parameter
ndim = len(original_model.spread_probs) + 1

# define the log-likelihood
def log_prob_fn(theta, sys, early_p=0.3, max_t=10):
    spread_probs, late_p = theta[:-1], theta[-1]
    
    if late_p > 1. or late_p < 0.:
        return -np.inf
    
    t = np.arange(max_t + 1)
    time_dists={
        "early": lymph.utils.fast_binomial_pmf(t, max_t, early_p),
        "late" : lymph.utils.fast_binomial_pmf(t, max_t, late_p)
    }
    
    return sys.marginal_log_likelihood(
        spread_probs, t_stages=["early", "late"], time_dists=time_dists
    )

<div class="alert alert-warning">
    Warning 

    The provided log-likelihood function won't be stored anywhere! It is not possible to store arbitrary python code in an HDF5 file and retrieve it automatically in a safe manner.
</div>

## Step 4: Do the sampling & store the results

In [12]:
# this chain might be a little too short, but for the demo it's OK
nstep, burnin = 200, 100

with Pool() as pool:
    original_sampler = lymph.utils.EnsembleSampler(
        ndim, log_prob_fn, args=[original_model], pool=pool
    )
    original_sampler.run_mcmc(nstep)
    
print(f"Acceptance fraction = {100 * np.mean(original_sampler.acceptance_fraction):.2f} %")

original_sampler.to_hdf5(
    filename=demo_hdf5_file, 
    groupname="original"
)

100%|██████████| 200/200 [05:11<00:00,  1.56s/it]

Acceptance fraction = 21.52 %


We can make sure the chain of samples is actually stored by trying to retrieve it from the HDF5 file directly:

In [13]:
chain = lymph.utils.EnsembleSampler.get_chain_from_hdf5(
    filename=demo_hdf5_file,
    groupname="original"
)
chain.shape

(200, 120, 12)

The first round has finished now. Let's see if we can repoduce all of that as intended.

## Do it all again

When we load a model instance from the HDF5 storage, all the settings, i.e. the graph, the diagnostic modalities and the loaded data, should still be the same as in the beginning. So let's check that with some `assert`s.

In [14]:
recovered_model = lymph.utils.system_from_hdf5(
    filename=demo_hdf5_file,
    groupname="original"
)

In [15]:
assert recovered_model.graph == graph, "Wrong graph!"
recovered_model.graph

{('tumor', 'primary'): ['I', 'II', 'III', 'IV'],
 ('lnl', 'I'): ['II'],
 ('lnl', 'II'): ['III'],
 ('lnl', 'III'): ['IV'],
 ('lnl', 'IV'): []}

In [16]:
assert recovered_model.modalities == diagnostic_spsn, "Wrong diagnostic modalities!"
recovered_model.modalities

{'MRI': [0.63, 0.81], 'PET': [0.86, 0.79]}

In [17]:
assert np.all(recovered_model.patient_data == synthetic_data), "Wrong data!"
recovered_model.patient_data

MRI                                                     PET         \
    contra                        ipsi                      contra          
         I     II    III     IV      I     II    III     IV      I     II   
0    False   True  False   True  False  False  False  False  False  False   
1     True   True  False  False   True  False  False  False  False   True   
2    False  False  False  False   True   True   True  False  False  False   
3    False   True   True   True  False  False  False  False  False  False   
4     True  False   True  False  False   True   True   True  False   True   
..     ...    ...    ...    ...    ...    ...    ...    ...    ...    ...   
195  False  False   True  False  False   True   True   True   True   True   
196  False  False   True   True   True  False  False  False  False  False   
197  False  False   True  False   True   True   True   True  False  False   
198   True   True  False  False  False   True  False   True  False  False   
199   True  False   True  False  False   True  False  False  False  False   

                                                 info  
                    ipsi                        tumor  
       III     IV      I     II    III     IV t_stage  
0    False  False  False  False  False  False   early  
1    False  False  False  False  False  False   early  
2    False  False   True   True  False   True    late  
3     True   True  False  False  False  False   early  
4    False  False  False  False   True   True   early  
..     ...    ...    ...    ...    ...    ...     ...  
195  False   True  False   True  False   True    late  
196  False  False  False   True  False  False   early  
197  False  False   True  False  False   True    late  
198  False  False  False   True   True  False   early  
199  False  False  False  False  False  False   early  

[200 rows x 17 columns]

The recovery worked! Since we want to do another sampling round which itself should be reproducable as well, we can immediately store the recovered model in a new group of the HDF5 file.

In [18]:
recovered_model.to_hdf5(
    filename=demo_hdf5_file,
    groupname="recovered"
)

Now for the `EnsembleSampler`. Don't forget that the `log_prob_fn` needs to be specified, as it cannot be retrieved from an HDF5 file.

In [19]:
recovered_sampler = lymph.utils.EnsembleSampler.from_hdf5(
    log_prob_fn=log_prob_fn,
    args=[recovered_model],
    filename=demo_hdf5_file,
    groupname="original"
)

In [20]:
recovered_sampler.ndim

12

In [21]:
recovered_sampler.nwalkers

120

The recovered sampler immediately has the correct number of dimensions and walkers and since we have passed the log-likelihood function to it already, we are ready for sampling again.

Note that this recovery process does not reassign the previously stored chain of samples to the recovered sampler, since that'd be confusing as we want to sample new ones now.

In [22]:
# this chain might be a little too short, but for the demo it's OK
nstep, burnin = 200, 100

with Pool() as pool:
    # pool is only for multiprocessing. Can also be passed as kwargs to the 
    # `from_hdf5` function
    recovered_sampler.pool = pool
    recovered_sampler.run_mcmc(nstep)
    
print(f"Acceptance fraction = {100 * np.mean(recovered_sampler.acceptance_fraction):.2f} %")

recovered_sampler.to_hdf5(
    filename=demo_hdf5_file, 
    groupname="recovered"
)

100%|██████████| 200/200 [08:33<00:00,  2.57s/it]

Acceptance fraction = 20.49 %


Beyond that one can of course use the [`h5py`](https://docs.h5py.org/en/stable/) package or `pandas`' implemented capabilities to interact with the HDF5 file format to store and retrieve even more information, like a description of what was done or what exactly the log-likelihood does.